In [13]:
#from datetime import datetime
#import pytz
#import json
#import urllib
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
base_url = 'http://meteo145.uibk.ac.at/'
station = 'innsbruck'
time = str(3)

url = base_url + station + '/' + time

In [35]:
df = pd.read_json(url)
df['time'] = [datetime(1970, 1, 1) + timedelta(milliseconds=ds) for ds in df['datumsec']]
df = df.set_index('time')
df = df.drop(columns='datumsec')
df.head()

,rr,dd,tp,p,tl,so,ff
time,,,,,,,
2019-10-21 07:20:00,0.0,240,8.9,948.4,13.7,0.0,3.3
2019-10-21 07:30:00,0.0,245,9.0,948.2,13.8,0.0,3.4
2019-10-21 07:40:00,0.0,258,9.0,948.6,13.6,0.0,3.9
2019-10-21 07:50:00,0.0,242,9.1,948.5,13.5,0.0,2.9
2019-10-21 08:00:00,0.0,233,9.3,948.2,13.8,0.0,2.7
